## 0, Download important libraries

In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1, Data Collection

#### As mentioned in report, I uitilized data from 2 different sources: 1, Data about hotels of Kyoto and nearby areas. I collected information about postal code, name, address of hotels. Then I integrated data into a database which contains 40 observations of 40 hotels from Kyoto and nearby areas. For a better analysis, I selected data of hotels from different areas and with different facilities. 2, Another source I used to explore and target recommended hotels across different venues is Foursquare location data. I imported it into a pandas dataframe for data exploration, visualization and modeling. The final database will have a combination of Foursquare location data and Kyoto nearby areas hotels data. The data was preprocessed before importing to this notebook.

In [29]:
# Read the Kyoto hotels data (Source: www.booking.com)
!wget -O Kyoto_Hotel.csv https://raw.githubusercontent.com/Klaire1229/Capstone-Project/master/Kyoto_Hotel.csv

--2020-05-03 20:54:51--  https://raw.githubusercontent.com/Klaire1229/Capstone-Project/master/Kyoto_Hotel.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2807 (2.7K) [text/plain]
Saving to: ‘Kyoto_Hotel.csv’

100%[======================================>] 2,807       --.-K/s   in 0s      

2020-05-03 20:54:51 (27.4 MB/s) - ‘Kyoto_Hotel.csv’ saved [2807/2807]



In [34]:
hotels = pd.read_csv('Kyoto_Hotel.csv')
hotels

,PostalCode,Hotel_Name,Address
0,600-8372,Hotel WBF Gojo Omiya,Shimogyo-ku Minamimonzencho 467
1,600-8029,Kyoto Rich Hotel,Shimogyo-ku Kawaramachi-dori Gojo-agaru 751
2,604-8365,Hotel M's Plus Shijo Omiya,Nakagyo-ku Nishikioomiyacho 114
3,600-8424,Nest Hotel Kyoto Shijo Karasuma,Shimogyo-ku Sanno-cho 551
4,605-0086,Hotel Alza Kyoto,Higashiyama-ku Benzaitencho 3-33
5,601-8017,HOTEL M's EST KYOTO STATION SOUTH,"Minami-ku Higashikujo Kitakarasumacho 7, 7-5, 7-6"
6,604-0902,The Ritz-Carlton Kyoto,Nakagyo-ku Kamogawa Nijo-ohashi Hotori
7,600-8022,GOOD NATURE HOTEL KYOTO,Shimogyo-ku Kawaramachi dori 2 chome Inaricho ...
8,605-0932,Four Seasons Hotel Kyoto,Higashiyama-ku Myohoin Maekawacho 445-3
9,604-0831,Tomoya Residence Hotel Kyoto,"Nakagyo-ku,Matsuya-cho 58"


In [35]:
hotels.shape

(40, 3)

In [36]:
# Use geopy library to get the latitude and longitude values of Kyoto city
address = 'Kyoto,Japan'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kyoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kyoto are 35.021041, 135.7556075.


In [37]:
# Assign based on postal code the latitude and longitode coordinates
geolocator = Nominatim(user_agent="my-application")
hotels['city_coord'] = hotels['PostalCode'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
hotels.head(5)

,PostalCode,Hotel_Name,Address,city_coord
0,600-8372,Hotel WBF Gojo Omiya,Shimogyo-ku Minamimonzencho 467,"(39.577961, -107.0140322)"
1,600-8029,Kyoto Rich Hotel,Shimogyo-ku Kawaramachi-dori Gojo-agaru 751,"(34.3422236, -109.753778)"
2,604-8365,Hotel M's Plus Shijo Omiya,Nakagyo-ku Nishikioomiyacho 114,"(35.0046953, 135.7489203)"
3,600-8424,Nest Hotel Kyoto Shijo Karasuma,Shimogyo-ku Sanno-cho 551,"(47.5062017, 17.3459677)"
4,605-0086,Hotel Alza Kyoto,Higashiyama-ku Benzaitencho 3-33,"(50.2235255, 14.4579288)"


In [38]:
hotels[['Latitude', 'Longitude']] = hotels['city_coord'].apply(pd.Series)

In [39]:
Hotels = hotels.drop(columns=['city_coord'])

In [41]:
Hotels 

,PostalCode,Hotel_Name,Address,Latitude,Longitude
0,600-8372,Hotel WBF Gojo Omiya,Shimogyo-ku Minamimonzencho 467,39.577961,-107.014032
1,600-8029,Kyoto Rich Hotel,Shimogyo-ku Kawaramachi-dori Gojo-agaru 751,34.342224,-109.753778
2,604-8365,Hotel M's Plus Shijo Omiya,Nakagyo-ku Nishikioomiyacho 114,35.004695,135.748920
3,600-8424,Nest Hotel Kyoto Shijo Karasuma,Shimogyo-ku Sanno-cho 551,47.506202,17.345968
4,605-0086,Hotel Alza Kyoto,Higashiyama-ku Benzaitencho 3-33,50.223526,14.457929
5,601-8017,HOTEL M's EST KYOTO STATION SOUTH,"Minami-ku Higashikujo Kitakarasumacho 7, 7-5, 7-6",34.371597,-109.723373
6,604-0902,The Ritz-Carlton Kyoto,Nakagyo-ku Kamogawa Nijo-ohashi Hotori,40.087305,-74.051894
7,600-8022,GOOD NATURE HOTEL KYOTO,Shimogyo-ku Kawaramachi dori 2 chome Inaricho ...,56.601386,14.182681
8,605-0932,Four Seasons Hotel Kyoto,Higashiyama-ku Myohoin Maekawacho 445-3,34.990327,135.775362
9,604-0831,Tomoya Residence Hotel Kyoto,"Nakagyo-ku,Matsuya-cho 58",40.088054,-74.052036


In [45]:


# Create a map of Kyoto using latitude and longitude values
map_kyoto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, hotels, street in zip(Hotels['Latitude'], Hotels['Longitude'], Hotels['Hotel_Name'], Hotels['Address']):
    label = '{}, {}'.format(street, hotels)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kyoto)  

map_kyoto



In [47]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'OK3ROAYOYVZ115CL1ENW4AJUXDZUD3V1LL1XPO2TEJRWRIXP' # your Foursquare ID
CLIENT_SECRET = 'XQBJ3JJF55SQ0HSL2M4H5AWCFVSONPAMK3K1J50ZEFRSU1PC' # your Foursquare Secret
VERSION = '20200501' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius=500
LIMIT=100

Your credentails:
CLIENT_ID: OK3ROAYOYVZ115CL1ENW4AJUXDZUD3V1LL1XPO2TEJRWRIXP
CLIENT_SECRET:XQBJ3JJF55SQ0HSL2M4H5AWCFVSONPAMK3K1J50ZEFRSU1PC


## 2, Data Exploration

In [58]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Hotel', 
                  'Hotel Latitude', 
                  'Hotel Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
hotels_venues = getNearbyVenues(names=Hotels['Hotel_Name'],
                                   latitudes=Hotels['Latitude'],
                                   longitudes=Hotels['Longitude']
                                  )

Hotel WBF Gojo Omiya
Kyoto Rich Hotel 
Hotel M's Plus Shijo Omiya 
Nest Hotel Kyoto Shijo Karasuma
Hotel Alza Kyoto
HOTEL M's EST KYOTO STATION SOUTH
The Ritz-Carlton Kyoto
GOOD NATURE HOTEL KYOTO
Four Seasons Hotel Kyoto
Tomoya Residence Hotel Kyoto
Kyoto Nanzenji Ryokan Yachiyo
YADORU KYOTO HANARE Eigetsu
Kyoto Yura Hotel MGallery
Luxury hotel SOWAKA
Daiwa Royal Hotel Grande Kyoto
The Westin Miyako Kyoto
The Thousand Kyoto
Kiraku Kyoto Aneyakoji
Grand Prince Hotel Kyoto
Kiraku Kyoto Higashiyama (Nazuna)
Kiraku Kyoto Zeniyacho
Aoi Philosophers Path Villa
BenTen Residences
Maana Kyoto
Yuzuya Ryokan
Suiran
the b kyoto shijo
Hotel Gran Ms Kyoto
Laon Inn Gion Nawate
APA Hotel Kyoto Ekimae
Kyotoya Kiyomizugojo
PARK HYATT KYOTO
Imu Hotel Kyoto
Hotel Resol Kyoto Kawaramachi Sanjo
Hyatt Regency Kyoto
TABITABI STAY _MOONLIGHT_
Kuraya Omiya Shimabara
Kiraku Kyoto Gion
Villa Sanjomuromachi KYOTO
THE GENERAL KYOTO Bukkoji Fuyacho


In [60]:
# Check the size of the resulting dataframe
print(hotels_venues.shape)
hotels_venues.head(10)

(832, 7)


,Hotel,Hotel Latitude,Hotel Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hotel M's Plus Shijo Omiya,35.004695,135.74892,Shomin (立ち飲み 庶民),35.003125,135.749275,Sake Bar
1,Hotel M's Plus Shijo Omiya,35.004695,135.74892,Gyoza no Ohsho (餃子の王将 四条大宮店),35.004095,135.748481,Chinese Restaurant
2,Hotel M's Plus Shijo Omiya,35.004695,135.74892,Hotel M's Plus Shijo-Omiya (ホテル エムズ・プラス四条大宮),35.003746,135.747741,Bed & Breakfast
3,Hotel M's Plus Shijo Omiya,35.004695,135.74892,京一,35.004243,135.748295,Chinese Restaurant
4,Hotel M's Plus Shijo Omiya,35.004695,135.74892,Fruit parlor Yaoiso (ヤオイソ),35.003621,135.750005,Café
5,Hotel M's Plus Shijo Omiya,35.004695,135.74892,とり伊,35.002743,135.750549,Japanese Restaurant
6,Hotel M's Plus Shijo Omiya,35.004695,135.74892,Bungalow (バンガロー),35.003519,135.752874,Beer Bar
7,Hotel M's Plus Shijo Omiya,35.004695,135.74892,串八 四条大宮店,35.003419,135.749279,Yakitori Restaurant
8,Hotel M's Plus Shijo Omiya,35.004695,135.74892,海鮮屋台 あみたつ (海鮮料理 網辰(あみたつ)),35.008480,135.746213,Sake Bar
9,Hotel M's Plus Shijo Omiya,35.004695,135.74892,蕎麦屋 樹庵,35.002520,135.746271,Soba Restaurant


In [61]:
# Check how many venues were returned for each hotel
hotels_venues.groupby('Hotel').count()

,Hotel Latitude,Hotel Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Hotel,,,,,,
BenTen Residences,7,7,7,7,7,7
Four Seasons Hotel Kyoto,36,36,36,36,36,36
Grand Prince Hotel Kyoto,3,3,3,3,3,3
Hotel Gran Ms Kyoto,100,100,100,100,100,100
Hotel M's Plus Shijo Omiya,57,57,57,57,57,57
Hotel Resol Kyoto Kawaramachi Sanjo,100,100,100,100,100,100
Imu Hotel Kyoto,1,1,1,1,1,1
Kiraku Kyoto Aneyakoji,2,2,2,2,2,2
Kiraku Kyoto Zeniyacho,2,2,2,2,2,2


#### The table above shows number of venues for each hotel. Hotels that close to center of ward and tourist attractions have the most facilities (e.g. Hotel Gran Ms Kyoto, Hotel Resol Kyoto Kawaramachi Sanjo, The Thousand Kyoto, PARK HYATT KYOTO ), compared to the hotels located further from center of ward and tourist attractions (e.g. Kiraku Kyoto Aneyakoji, Grand Prince Hotel Kyoto, Imu Hotel Kyoto ).


In [62]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(hotels_venues['Venue Category'].unique())))

There are 161 uniques categories.


In [63]:
#### 2.2. Analyze each hotel
# one hot encoding
hotels_onehot = pd.get_dummies(hotels_venues[['Venue Category']], prefix="", prefix_sep="")

# add hotel column back to dataframe
hotels_onehot['Hotel'] = hotels_venues['Hotel'] 

# move hotel column to the first column
fixed_columns = [hotels_onehot.columns[-1]] + list(hotels_onehot.columns[:-1])
hotels_onehot = hotels_onehot[fixed_columns]

hotels_onehot.head(5)


,Yoshoku Restaurant,American Restaurant,Antique Shop,Arcade,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Bus Stop,Café,Camera Store,Canal,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kaiseki Restaurant,Kushikatsu Restaurant,Lake,Liquor Store,Market,Memorial Site,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Museum,Music Store,Music Venue,Nabe Restaurant,Noodle House,Office,Okonomiyaki Restaurant,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Perfume Shop,Pet Café,Pharmacy,Pizza Place,Platform,Playground,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Recording Studio,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Club,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shrine,Snack Place,Soba Restaurant,Soccer Field,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Taco Place,Takoyaki Place,Tapas Restaurant,Tea Room,Temple,Tempura Restaurant,Theater,Theme Restaurant,Tonkatsu Restaurant,Trail,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Video Store,Wagashi Place,Water Park,Whisky Bar,Women's Store,Yakitori Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Hotel M's Plus Shijo Omiya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Hotel M's Plus Shijo Omiya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Hotel M's Plus Shijo Omiya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Hotel M's Plus Shijo Omiya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Hotel M's Plus Shijo Omiya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
# Examine the new dataframe size
hotels_onehot.shape

(832, 161)

In [65]:
# Group rows by hotel and by taking the mean of the frequency of occurrence of each category
hotels_grouped = hotels_onehot.groupby('Hotel').mean().reset_index()
hotels_grouped

,Hotel,Yoshoku Restaurant,American Restaurant,Antique Shop,Arcade,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Buddhist Temple,Buffet,Bus Stop,Café,Camera Store,Canal,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kaiseki Restaurant,Kushikatsu Restaurant,Lake,Liquor Store,Market,Memorial Site,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Museum,Music Store,Music Venue,Nabe Restaurant,Noodle House,Office,Okonomiyaki Restaurant,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Perfume Shop,Pet Café,Pharmacy,Pizza Place,Platform,Playground,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Recording Studio,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Club,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shrine,Snack Place,Soba Restaurant,Soccer Field,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Taco Place,Takoyaki Place,Tapas Restaurant,Tea Room,Temple,Tempura Restaurant,Theater,Theme Restaurant,Tonkatsu Restaurant,Trail,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Video Store,Wagashi Place,Water Park,Whisky Bar,Women's Store,Yakitori Restaurant
0,BenTen Residences,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.00,0.142857,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000
1,Four Seasons Hotel Kyoto,0.00,0.027778,0.000000,0.00,0.027778,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.027778,0.055556,0.000,0.055556,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.027778,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.027778,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.027778,0.0

#### The table above shows available venues close to each hotel. There are wide range of restaurants (Yoshoku, American, Chinese, Donburi, Dim Sum.etc), Art museum, Antique Shop, Beer bar, Camera store, Flower shop, Ice Cream Shop, grocery stores, Train station and so on.

In [67]:
# Confirm the new size
hotels_grouped.shape

(21, 161)

In [68]:
# Print each hotel along with the top 5 most common venues
num_top_venues = 5

for Code in hotels_grouped['Hotel']:
    print("Hotel:", Code)
    temp = hotels_grouped[hotels_grouped['Hotel'] == Code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Hotel: BenTen Residences
                venue  freq
0           Drugstore  0.14
1    Doner Restaurant  0.14
2    Greek Restaurant  0.14
3  Miscellaneous Shop  0.14
4      Hardware Store  0.14


Hotel: Four Seasons Hotel Kyoto
                 venue  freq
0  Japanese Restaurant  0.08
1                 Café  0.08
2        Wagashi Place  0.06
3   Italian Restaurant  0.06
4      Buddhist Temple  0.06


Hotel: Grand Prince Hotel Kyoto
                venue  freq
0           Rest Area  0.33
1      Breakfast Spot  0.33
2         Flower Shop  0.33
3  Yoshoku Restaurant  0.00
4          Playground  0.00


Hotel: Hotel Gran Ms Kyoto
                 venue  freq
0  Japanese Restaurant  0.07
1          Coffee Shop  0.07
2                 Café  0.07
3     Ramen Restaurant  0.05
4               Bakery  0.04


Hotel: Hotel M's Plus Shijo Omiya 
                 venue  freq
0    Convenience Store  0.18
1     Ramen Restaurant  0.07
2  Yakitori Restaurant  0.05
3                 Café  0.05
4   Chinese 

#### We can observe for each hotel the top 5 most common venues. Hotels from central area have in particular facilities, such as: local and foreign restaurants, cafe, bar, bakery, grocery stores along with tourist attractions: Nineizaka (Ninenzaka), Hōkan-ji Temple, The Museum of Kyoto, Higashiyama Jisho-ji, Heian Shrine and so on.

In [69]:
#  Load into a pandas dataframe and sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
# Create the new dataframe and display the top 10 venues for each hotel
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Hotel']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [72]:
# Show for each hotel the most common venue
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Hotel'] = hotels_grouped['Hotel']

for ind in np.arange(hotels_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(hotels_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head(10)

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BenTen Residences,Hardware Store,Soccer Field,Doner Restaurant,Drugstore,Greek Restaurant,Supermarket,Miscellaneous Shop,French Restaurant,Fountain,Food Truck
1,Four Seasons Hotel Kyoto,Japanese Restaurant,Café,Bus Stop,Buddhist Temple,Italian Restaurant,Wagashi Place,Convenience Store,Sake Bar,Garden,Tea Room
2,Grand Prince Hotel Kyoto,Flower Shop,Rest Area,Breakfast Spot,Yakitori Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop
3,Hotel Gran Ms Kyoto,Café,Coffee Shop,Japanese Restaurant,Ramen Restaurant,Chinese Restaurant,Bakery,French Restaurant,Inn,Bar,Pub
4,Hotel M's Plus Shijo Omiya,Convenience Store,Ramen Restaurant,Yakitori Restaurant,Café,Chinese Restaurant,Sake Bar,Fast Food Restaurant,Bed & Breakfast,Bookstore,Liquor Store
5,Hotel Resol Kyoto Kawaramachi Sanjo,Japanese Restaurant,Coffee Shop,Ramen Restaurant,Café,Sake Bar,Chinese Restaurant,Sushi Restaurant,Historic Site,Inn,Pub
6,Imu Hotel Kyoto,Mexican Restaurant,Gastropub,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Fast Food Restaurant
7,Kiraku Kyoto Aneyakoji,Playground,Farmers Market,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop
8,Kiraku Kyoto Zeniyacho,Bus Stop,Yakitori Restaurant,Discount Store,Garden,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop
9,Kyotoya Kiyomizugojo,Sandwich Place,Yakitori Restaurant,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop


## 3, Data Modelling

In [96]:
# Run k-means to cluster the neighborhood into 7 clusters
# Set number of clusters
kclusters = 7
hotels_grouped_clustering = hotels_grouped.drop('Hotel', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hotels_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([0, 0, 5, 0, 0, 0, 3, 1, 4, 2, 6, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
      dtype=int32)

In [98]:
# Add labels
#venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
#hotels_merged = Hotels
#hotels_merged.rename(columns={'Hotel_Name': 'Hotel'}, inplace=True)

In [109]:
# merge kyoto_grouped with kyoto to add latitude/longitude for each neighborhood
#hotels_merged = hotels_merged.join(venues_sorted.set_index('Hotel'), on='Hotel')
#hotels_merged.head()



In [100]:
hotels_merged = hotels_merged.fillna(0)
hotels_merged['Cluster Labels']=hotels_merged['Cluster Labels'].astype(int)

In [101]:
# Visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hotels_merged['Latitude'], hotels_merged['Longitude'], hotels_merged['Hotel'], hotels_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### The map shows the cluster grouping mode. It can be noticed that hotels with common venues and many tourist attractions are grouped together, compared with hotels located in less crowded areas and with fewer facilities and venues

In [102]:
#  Examine each cluster
# 1st cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 0, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hotel WBF Gojo Omiya,0,0,0,0,0,0,0,0,0,0,0
1,Kyoto Rich Hotel,0,0,0,0,0,0,0,0,0,0,0
2,Hotel M's Plus Shijo Omiya,0,Convenience Store,Ramen Restaurant,Yakitori Restaurant,Café,Chinese Restaurant,Sake Bar,Fast Food Restaurant,Bed & Breakfast,Bookstore,Liquor Store
3,Nest Hotel Kyoto Shijo Karasuma,0,0,0,0,0,0,0,0,0,0,0
4,Hotel Alza Kyoto,0,0,0,0,0,0,0,0,0,0,0
5,HOTEL M's EST KYOTO STATION SOUTH,0,0,0,0,0,0,0,0,0,0,0
6,The Ritz-Carlton Kyoto,0,Ice Cream Shop,Italian Restaurant,Brewery,Mexican Restaurant,Pizza Place,Liquor Store,Pharmacy,Discount Store,Diner,Restaurant
7,GOOD NATURE HOTEL KYOTO,0,0,0,0,0,0,0,0,0,0,0
8,Four Seasons Hotel Kyoto,0,Japanese Restaurant,Café,Bus Stop,Buddhist Temple,Italian Restaurant,Wagashi Place,Convenience Store,Sake Bar,Garden,Tea Room
9,Tomoya Residence Hotel Kyoto,0,Sandwich Place,Brewery,Ice Cream Shop,Diner,BBQ Joint,Pizza Place,Italian Restaurant,Taco Place,Café,Juice Bar


In [103]:
# 2nd cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 1, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Kiraku Kyoto Aneyakoji,1,Playground,Farmers Market,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop
35,TABITABI STAY _MOONLIGHT_,1,Playground,Farmers Market,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop
38,Villa Sanjomuromachi KYOTO,1,Playground,Farmers Market,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop


In [104]:
# 3rd cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 2, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Kyotoya Kiyomizugojo,2,Sandwich Place,Yakitori Restaurant,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop


In [105]:
# 4th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 3, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Imu Hotel Kyoto,3,Mexican Restaurant,Gastropub,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Fast Food Restaurant


In [106]:
# 5th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 4, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Kiraku Kyoto Zeniyacho,4,Bus Stop,Yakitori Restaurant,Discount Store,Garden,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop


In [107]:
# 6th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 5, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Grand Prince Hotel Kyoto,5,Flower Shop,Rest Area,Breakfast Spot,Yakitori Restaurant,Furniture / Home Store,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop


In [108]:
# 7th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 6, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Laon Inn Gion Nawate,6,History Museum,Historic Site,German Restaurant,Bakery,Soccer Field,Palace,French Restaurant,Fountain,Food Truck,Food Court
